In [1]:
import datetime, os, json, csv
from collections import Counter

In [2]:
nowtime = f'{datetime.datetime.now():%Y%m%d}'
one_week_ago = f'{datetime.datetime.now() - datetime.timedelta(days=7):%Y%m%d}'
print(nowtime)

20201124


In [3]:
os.makedirs('./resources/ch09-twittercookbook/'+ str(nowtime), exist_ok = True)

## API 設定

In [4]:
import twitter

def oauth_login():

    CONSUMER_KEY = 'KOThudX3C7brplSrDa7vSVM2T' #API key 
    CONSUMER_SECRET = 'Fx5shLni0RcK1mPvLkANVbzSmGYK3pUIM0ZfiebpdzCykvVNuA' #API key select
    OAUTH_TOKEN = '109162821-Kh4y6qfclwVBKr8cz978DU7Fz1F0nGs7HiZH34PV'
    OAUTH_TOKEN_SECRET = 'rXd5DKwiAiKQJIBY6o7vH66Od5mQXrHEBP1GEVqoBY9bE'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

# Sample usage
twitter_api = oauth_login()    

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

print(twitter_api)

## ツィートを検索する

In [5]:
from urllib.parse import unquote

# XXX: Set this variable to a trending topic, 
# or anything else for that matter. The example query below
# was a trending topic when this content was being developed
# and is used throughout the remainder of this chapter.

#q = '出生前診断' #11 2020/9/3　 word 448　9/8 483　9/15 600　9/17 600 9/24 492 9/27 365
#q = 'NIPT' #35  word 427 9/8  358 9/15 368 9/17 343 9/24 310　9/27 306
#q = 'クアトロテスト' #0  word 18 9/8 15 9/15 14 9/17 12 9/24 18 9/27 14
#q = '羊水検査' #9/27 = 113 #q = 'コンバインドテスト'　#9/27 5　# q= 'トリプルマーカーテスト' #9/27 1
q = 'AYA世代'

count = 100

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets

search_results = twitter_api.search.tweets(q=q, count=count,lang='ja')

statuses = search_results['statuses']


# Iterate through 5 more batches of results by following the cursor

for _ in range(5):
    print("Length of statuses", len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError as e: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in unquote(next_results[1:]).split("&") ])
    
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

# Show one sample search result by slicing the list...
#print(json.dumps(statuses[0], indent=1,ensure_ascii=False))
print(one_week_ago,'から',nowtime,'までの、キーワード「',
      q,'」を含むツィートの件数',len(statuses))

Length of statuses 100
Length of statuses 200
Length of statuses 300
Length of statuses 400
Length of statuses 408
20201117 から 20201124 までの、キーワード「 AYA世代 」を含むツィートの件数 408


## ツィートを検索する（関数）

In [6]:
def twitter_search(twitter_api, q, max_results=100, **kw):

    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets and 
    # https://dev.twitter.com/docs/using-search for details on advanced 
    # search criteria that may be useful for keyword arguments
    
    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets    
    search_results = twitter_api.search.tweets(q=q, count=100, **kw)
    
    statuses = search_results['statuses']
    
    # Iterate through batches of results by following the cursor until we
    # reach the desired number of results, keeping in mind that OAuth users
    # can "only" make 180 search queries per 15-minute interval. See
    # https://dev.twitter.com/docs/rate-limiting/1.1/limits
    # for details. A reasonable number of results is ~1000, although
    # that number of results may not exist for all queries.
    
    # Enforce a reasonable limit
    max_results = min(1000, max_results)
    
    for _ in range(10): # 10*100 = 1000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break
            
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        
        if len(statuses) > max_results: 
            break
            
    return statuses

# Sample usage

twitter_api = oauth_login()

#q = "出生前診断"
results = twitter_search(twitter_api, q, max_results=10,lang='ja')
        
# Show one sample search result by slicing the list...
print(json.dumps(results[0], indent=1,ensure_ascii=False))
#print(len(statuses))

{
 "created_at": "Tue Nov 24 05:29:57 +0000 2020",
 "id": 1331107769181638656,
 "id_str": "1331107769181638656",
 "text": "RT @3xLqH0zFfvo1VdY: GQJAPAN受賞式\n鈴木編集長のコメント文字起こし\n\nSixTONESに物凄く期待している\n今まで日本に無かったタイプのグループ\n新しい世代を代表してる。国際的な普遍性を持っていく。それを正面から見据えて音楽的にそれを目指してる。大変…",
 "truncated": false,
 "entities": {
  "hashtags": [],
  "symbols": [],
  "user_mentions": [
   {
    "screen_name": "3xLqH0zFfvo1VdY",
    "name": "緋💎🇯🇵自ﾂｲいいね欄",
    "id": 1223542756364050433,
    "id_str": "1223542756364050433",
    "indices": [
     3,
     19
    ]
   }
  ],
  "urls": []
 },
 "metadata": {
  "iso_language_code": "ja",
  "result_type": "recent"
 },
 "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>",
 "in_reply_to_status_id": null,
 "in_reply_to_status_id_str": null,
 "in_reply_to_user_id": null,
 "in_reply_to_user_id_str": null,
 "in_reply_to_screen_name": null,
 "user": {
  "id": 1275783373235949568,
  "id_str": "1275783373235949568",
  "name": "makaro

## JSONデータをテキストで保存する

In [7]:
import io, json

In [8]:
def save_json(filename, data):
    with io.open('resources/ch09-twittercookbook/{0}/{1}.json'.format(nowtime,filename), 
                 'w', encoding='utf-8') as f:
        f.write(json.dumps(data, ensure_ascii=False))

def load_json(filename):
    with io.open('resources/ch09-twittercookbook/{0}/{1}.json'.format(nowtime,filename), 
                 encoding='utf-8') as f:
        return f.read()

# Sample usage

#q = '出生前診断'

twitter_api = oauth_login()
results = twitter_search(twitter_api, q, max_results=10,lang='ja')

save_json(q, results)
results = load_json(q)
#print(json.dumps(results, indent=1,ensure_ascii=False))

## 保存したJSONを開く

In [9]:
when = str(
    #20200903 #20200908 #20200915 #20200917　#20200924 
    nowtime
)
statuses = json.loads(
    open('resources/ch09-twittercookbook/'+when +'/'+str(q) + '.json').read())


## テキスト、ハッシュタグ、screen namesを抽出する

In [10]:
status_texts = [ status['text'] 
                 for status in statuses ]

screen_names = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

# Compute a collection of all words from all tweets
words = [ w 
          for t in status_texts 
              for w in t.split() ]

# Explore the first 5 items for each...
#print('texts: {}\n screen_names: {}\n hashtags: {}\n words: {}'.format(
    #len(statuses),len(screen_names),len(hashtags),len(words)))
print(json.dumps(status_texts[0:5], indent=1,ensure_ascii=False))
print(json.dumps(screen_names[0:5], indent=1,ensure_ascii=False))
print(json.dumps(hashtags[0:5], indent=1,ensure_ascii=False))
#print(json.dumps(words[0:5], indent=1,ensure_ascii=False))

[
 "RT @3xLqH0zFfvo1VdY: GQJAPAN受賞式\n鈴木編集長のコメント文字起こし\n\nSixTONESに物凄く期待している\n今まで日本に無かったタイプのグループ\n新しい世代を代表してる。国際的な普遍性を持っていく。それを正面から見据えて音楽的にそれを目指してる。大変…",
 "今朝は念願叶いケアホーム西大井こうほうえんを視察して参りました！\n\n江東区では新砂保育園と高齢者施設を運営されている法人です。\n\n多世代型交流ができる施設としては勿論、廃校後の学校施設跡地活用としても大変参考になりました。\n\n西本… https://t.co/TZTbKVClCB",
 "RT @uedareiko: 【ケアホーム西大井こうほうえん視察】\n旧品川区立原小学校舎を活用してサービス付高齢者向け住宅、認可保育園、高齢者交流ひろば、体育館はスポーツと地域に開かれた多世代利用施設の視察を、WM議員の大先輩西本貴子品川区議のご案内、社会福祉士の三戸 安弥江東…",
 "RT @PinkRing5: Pink Ringより\n\n全国のAYA世代がん体験者の皆様へ\n（がん種・現在の年齢不問）マスクプレゼントのお知らせ🎀\n\n拡散大歓迎✨\n\n詳細は、HPをご覧下さい！\n\nhttps://t.co/Ea0fYr1QCX https://t.co/mFUu…",
 "RT @GenerationAya: 【12月5日(土) 9:00スタート】\n《AYA世代》をメインにしたプログラムです。\n\n＊参加無料 (応援チケットあり) ⬇︎\nhttps://t.co/ouxHHZMRN8\n\nAYAまつり専用プログラムサイト⬇︎\nhttps://t.co…"
]
[
 "3xLqH0zFfvo1VdY",
 "uedareiko",
 "PinkRing5",
 "GenerationAya",
 "Sponichi_Tfoto"
]
[
 "山瀬慎之助",
 "井上広大",
 "aya世代",
 "がん",
 "AYAweek"
]


# LDA で教師なし分類
https://qiita.com/nakashimayugo/items/d42475c9ddd1d5697c1d

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame({'contents':status_texts})
df.head()

,contents
0,RT @3xLqH0zFfvo1VdY: GQJAPAN受賞式\n鈴木編集長のコメント文字起...
1,今朝は念願叶いケアホーム西大井こうほうえんを視察して参りました！\n\n江東区では新砂保育園...
2,RT @uedareiko: 【ケアホーム西大井こうほうえん視察】\n旧品川区立原小学校舎を...
3,RT @PinkRing5: Pink Ringより\n\n全国のAYA世代がん体験者の皆様...
4,RT @GenerationAya: 【12月5日(土) 9:00スタート】\n《AYA世代...


In [13]:
df['contents'].unique()

array(['RT @3xLqH0zFfvo1VdY: GQJAPAN受賞式\n鈴木編集長のコメント文字起こし\n\nSixTONESに物凄く期待している\n今まで日本に無かったタイプのグループ\n新しい世代を代表してる。国際的な普遍性を持っていく。それを正面から見据えて音楽的にそれを目指してる。大変…',
       '今朝は念願叶いケアホーム西大井こうほうえんを視察して参りました！\n\n江東区では新砂保育園と高齢者施設を運営されている法人です。\n\n多世代型交流ができる施設としては勿論、廃校後の学校施設跡地活用としても大変参考になりました。\n\n西本… https://t.co/TZTbKVClCB',
       'RT @uedareiko: 【ケアホーム西大井こうほうえん視察】\n旧品川区立原小学校舎を活用してサービス付高齢者向け住宅、認可保育園、高齢者交流ひろば、体育館はスポーツと地域に開かれた多世代利用施設の視察を、WM議員の大先輩西本貴子品川区議のご案内、社会福祉士の三戸 安弥江東…',
       'RT @PinkRing5: Pink Ringより\n\n全国のAYA世代がん体験者の皆様へ\n（がん種・現在の年齢不問）マスクプレゼントのお知らせ🎀\n\n拡散大歓迎✨\n\n詳細は、HPをご覧下さい！\n\nhttps://t.co/Ea0fYr1QCX https://t.co/mFUu…',
       'RT @GenerationAya: 【12月5日(土) 9:00スタート】\n《AYA世代》をメインにしたプログラムです。\n\n＊参加無料 (応援チケットあり) ⬇︎\nhttps://t.co/ouxHHZMRN8\n\nAYAまつり専用プログラムサイト⬇︎\nhttps://t.co…',
       'Samplingmaster MEGAとかAYAっていうのはつまり「そういうこと」なんだと思います。この人たちFM音源現役世代ですし。恐ろしい！',
       'RT @Sponichi_Tfoto: 試合前に同世代の ＃山瀬慎之助 さんと談笑する ＃井上広大 さんです。 https://t.co/YTFHBlnXd5',
       '@aya__ss_ ホントなんだろ涙が気づいたら出てる\u200dｗｗｗ

In [14]:
#必要なライブラリをimport
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import joblib
from tqdm import tqdm_notebook as tqdm
import numpy as np

In [15]:
#文章×単語行列に変換
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b") 
#訓練データ用にBow表現に。
X = vectorizer.fit_transform(df["contents"]) 

In [16]:
#単語番号と単語を紐づけ
index2word = {}
for word, word_id in tqdm(vectorizer.vocabulary_.items()):
    index2word[word_id] = word

/home/lcj001/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
#3分類でインスタンス作成。モデルの作成
lda = LatentDirichletAllocation(n_components=10, n_jobs=-1, verbose=1)
X_transformed = lda.fit_transform(X)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [18]:
#各トピックの特徴単語を抽出
for t in range(lda.n_components):
    # トピック t における、単語の出現確率を lda.components_[t] で取得
    # np.argsort で昇順に並び替えた順の index を取得、[::-1] で降順に並び替え
    indices = np.argsort(lda.components_[t])[::-1]
    print("Topic #{} ----------------------------------------".format(t+1))
    for word_id in indices[:10]:
        print(index2word[word_id])
    print()

Topic #1 ----------------------------------------
rt
co
https
t
ayaまつり
15
一色紗英
ර
がん
ayaweek

Topic #2 ----------------------------------------
rt
また
と
賛成
命を守る為
が高齢者の方です
不要不急の外出をお控え下さい
2時間以内
高齢者の方や基礎疾患をお持ちの方は
高齢者へ不要不急の外出自粛呼びかけ

Topic #3 ----------------------------------------
https
co
t
rt
訳
岩波書店
グリーン
世代の癌患者
さよならを待つふたりのために
若年成人

Topic #4 ----------------------------------------
rt
co
https
t
火
よろしくお願いいたします
tmcmc_pr
申込期限延長
申込期限を11月24日
オンライン交流イベントを開催いたします

Topic #5 ----------------------------------------
rt
makiniki_1209
aya_en_niki
そうです私はかなり48オタクです
あうんのキス聞けて最高でしたね
シャングリラ
アニメの話も盛り上がれて最高
ご視聴いただきありがとうございました
幽白ぜひ見て欲しいな
でも

Topic #6 ----------------------------------------
the
rt
to
t
a
月
に配信決定
のartist
30
rising

Topic #7 ----------------------------------------
rt
co
t
https
ω
sayano294692
w
و
monkichiruo27
aya_n_0419

Topic #8 ----------------------------------------
t
co
https
rt
aya世代
9
12月5日
応援チケットあり
土
参加無料

Topic #9 ----------------------------------------
rt
co
https
t
たったひとつの願い
aya世

# lda
https://pypi.org/project/lda/

In [19]:
#ライブラリのimport
import lda
import lda.datasets

In [20]:
#インスタンスなど
X = lda.datasets.load_reuters()
vocab = lda.datasets.load_reuters_vocab()
titles = lda.datasets.load_reuters_titles()

In [21]:
#上のscikit-learnで作った単語行列から、欲しいモノを取ってくる。
vect = vectorizer.get_feature_names()

In [22]:
#3分類でモデルの生成
model = lda.LDA(n_topics=8, n_iter=1500, random_state=1, alpha = 10)
model.fit(X)  # model.fit_transform(X) is also available

INFO:lda:n_documents: 395
INFO:lda:vocab_size: 4258
INFO:lda:n_words: 84010
INFO:lda:n_topics: 8
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -921112
INFO:lda:<10> log likelihood: -720542
INFO:lda:<20> log likelihood: -697360
INFO:lda:<30> log likelihood: -689773
INFO:lda:<40> log likelihood: -685235
INFO:lda:<50> log likelihood: -682847
INFO:lda:<60> log likelihood: -681275
INFO:lda:<70> log likelihood: -680055
INFO:lda:<80> log likelihood: -679501
INFO:lda:<90> log likelihood: -679474
INFO:lda:<100> log likelihood: -679741
INFO:lda:<110> log likelihood: -677891
INFO:lda:<120> log likelihood: -677823
INFO:lda:<130> log likelihood: -676915
INFO:lda:<140> log likelihood: -676662
INFO:lda:<150> log likelihood: -676572
INFO:lda:<160> log likelihood: -675861
INFO:lda:<170> log likelihood: -675647
INFO:lda:<180> log likelihood: -675380
INFO:lda:<190> log likelihood: -675961
INFO:lda:<200> log likelihood: -674904
INFO:lda:<210> log likelihood: -675020
INFO:lda:<220> log likelihood: -67

In [23]:
#単語抽出
for t in range(model.n_topics):
    indices = np.argsort(model.topic_word_[t])[::-1]
    print("Topic #{} ----------------------------------------".format(t+1))
    for word_id in indices[:10]:
        print(index2word[word_id])
    print()

Topic #1 ----------------------------------------
71で出場させて頂きました
_suzukitakako_
11月21日はポケモン
aya_n_0419
djpmx_official
3
2013年
14で計算すると
2020
lei260

Topic #2 ----------------------------------------
0
0ae17v5b7g
0e1rkaaxuq
15歳息子に昔のガラケーについて説明していて
38発の中央特快東京行きは高尾と西八王子からの高校生の通学客でホームや車内が混んでいた
25より
12月29日
10年生存率の統計が更新されるたび思うのは
3xlqh0zffvo1vdy
14で計算すると

Topic #3 ----------------------------------------
00スタート
20歳以降の女性に子宮頸がんや乳がんが急増する傾向にある
3qe183m6qn
1ページ読み込むたびに1コマ
aya_ni_3
aya_t_ga
0
13ts_44e
aya祭りと違う回線使うので
2世代思い入れあるからな

Topic #4 ----------------------------------------
12月5日まで
2021年には25歳
2021年には27歳
14歳の小児と15
3ds
6xmlikbprx
15時までお
7問正解です
9
2013年

Topic #5 ----------------------------------------
15
39歳の思春期
aya世代がん患者支援の一環として
3み
bとも第1
glayさんの
gqjapan受賞式
k
magurin4570
0ae17v5b7g

Topic #6 ----------------------------------------
0zcpk6lzhw
12月5日のyoutube配信ご質問いただきました
40歳前後の方は2つか3つしか採卵できないこともあるらしく
50代の20年生存率とかが本当は知りたいんだ
art__mky
_aya_jcr
birthday
ea0fyr1qcx
10年生存率の統計が更新されるたび思うのは
10問中

Topic #7 ------